<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1">Описание проекта</a></span><ul class="toc-item"><li><span><a href="#Дополнительное-задание:" data-toc-modified-id="Дополнительное-задание:-1.1">Дополнительное задание:</a></span><ul class="toc-item"><li><span><a href="#Описание-этапа-обработки" data-toc-modified-id="Описание-этапа-обработки-1.1.1">Описание этапа обработки</a></span></li><li><span><a href="#Описание-данных" data-toc-modified-id="Описание-данных-1.1.2">Описание данных</a></span></li></ul></li></ul></li><li><span><a href="#Реализация-задания" data-toc-modified-id="Реализация-задания-2">Реализация задания</a></span><ul class="toc-item"><li><span><a href="#Импорт-библиотек.-Загрузка-данных" data-toc-modified-id="Импорт-библиотек.-Загрузка-данных-2.1">Импорт библиотек. Загрузка данных</a></span></li><li><span><a href="#Анализ-данных" data-toc-modified-id="Анализ-данных-2.2">Анализ данных</a></span><ul class="toc-item"><li><span><a href="#1.-data_arc---данные-об-электродах" data-toc-modified-id="1.-data_arc---данные-об-электродах-2.2.1">1. data_arc - данные об электродах</a></span></li><li><span><a href="#2.-data_temp---данные-о-температуре:" data-toc-modified-id="2.-data_temp---данные-о-температуре:-2.2.2">2. data_temp - данные о температуре:</a></span></li></ul></li></ul></li></ul></div>

# Описание проекта

Чтобы оптимизировать производственные расходы, металлургический комбинат ООО «Так закаляем сталь» решил уменьшить потребление электроэнергии на этапе обработки стали. Вам предстоит построить модель, которая предскажет температуру стали.

## Дополнительное задание:

Вам поручили изучить характеристики процесса обработки стали. Для каждого значения столбца key найдите:

- [x] Длительность времени между первым и последним замером температуры.
- [x] Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов.
- [x] Количество запусков нагрева электродами.
- [x] Среднее соотношение потребления активной и реактивной мощности.

По всем полученным столбцам вычислите статистики: средние, минимальные и максимальные значения, медиану и величины 25%- и 75%-квартилей.

### Описание этапа обработки

Сталь обрабатывают в металлическом ковше вместимостью около 100 тонн. Чтобы ковш выдерживал высокие температуры, изнутри его облицовывают огнеупорным кирпичом. Расплавленную сталь заливают в ковш и подогревают до нужной температуры графитовыми электродами. Они установлены в крышке ковша. 

<img src="https://news.store.rambler.ru/img/d52ddd85719025e93d2be214d6b49da6" alt="Drawing" style="width: 600px;"/>

Из сплава выводится сера (десульфурация), добавлением примесей корректируется химический состав и отбираются пробы. Сталь легируют — изменяют её состав — подавая куски сплава из бункера для сыпучих материалов или проволоку через специальный трайб-аппарат (англ. tribe, «масса»).

Перед тем как первый раз ввести легирующие добавки, измеряют температуру стали и производят её химический анализ. Потом температуру на несколько минут повышают, добавляют легирующие материалы и продувают сплав инертным газом. Затем его перемешивают и снова проводят измерения. Такой цикл повторяется до достижения целевого химического состава и оптимальной температуры плавки.

Тогда расплавленная сталь отправляется на доводку металла или поступает в машину непрерывной разливки. Оттуда готовый продукт выходит в виде заготовок-слябов (англ. *slab*, «плита»).

### Описание данных

Данные состоят из файлов, полученных из разных источников:

- `data_arc.csv` — данные об электродах;
- `data_bulk.csv` — данные о подаче сыпучих материалов (объём);
- `data_bulk_time.csv` *—* данные о подаче сыпучих материалов (время);
- `data_gas.csv` — данные о продувке сплава газом;
- `data_temp.csv` — результаты измерения температуры;
- `data_wire.csv` — данные о проволочных материалах (объём);
- `data_wire_time.csv` — данные о проволочных материалах (время).

Во всех файлах столбец `key` содержит номер партии. В файлах может быть несколько строк с одинаковым значением `key`: они соответствуют разным итерациям обработки.

# Реализация задания

## Импорт библиотек. Загрузка данных

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

import warnings

In [2]:
pd.options.mode.chained_assignment = None  

In [3]:
data_arc = pd.read_csv('final_steel/data_arc.csv')
data_bulk = pd.read_csv('final_steel/data_bulk.csv')
data_bulk_time = pd.read_csv('final_steel/data_bulk_time.csv')
data_gas = pd.read_csv('final_steel/data_gas.csv')
data_temp = pd.read_csv('final_steel/data_temp.csv')
data_wire = pd.read_csv('final_steel/data_wire.csv')
data_wire_time = pd.read_csv('final_steel/data_wire_time.csv')

## Анализ данных

### 1. data_arc - данные об электродах

Для удобства работы с таблицами, переименуем столбцы:

In [4]:
def rename_columns(data, new_names=[], rename=True):
    if rename == False:
        new_names=[]
        for col in data.columns:
            new_names.append(str.lower('_'.join(col.split(' '))))
        data.columns = new_names
    else:
        data.columns = new_names

Подготовим новые названия для признаков. Те признаки, которые уже написаны латиницей просто склеим вместе через "_":

In [5]:
data_arc_cols = ['key', 'start_heating', 'end_heating', 'active_power', 'reactive_power']
data_gas_cols = ['key', 'gas']
data_temp_cols = ['key', 'measure_time', 'temperature']

Переименуем:

In [6]:
rename_columns(data_arc, data_arc_cols, rename=True)
rename_columns(data_bulk, rename=False)
rename_columns(data_bulk_time, rename=False)
rename_columns(data_gas, data_gas_cols, rename=True)
rename_columns(data_temp, data_temp_cols, rename=True)
rename_columns(data_wire, rename=False)
rename_columns(data_wire_time, rename=False)

In [7]:
data_arc.head(3)

,key,start_heating,end_heating,active_power,reactive_power
0,1,2019-05-03 11:02:14,2019-05-03 11:06:02,0.976059,0.687084
1,1,2019-05-03 11:07:28,2019-05-03 11:10:33,0.805607,0.520285
2,1,2019-05-03 11:11:44,2019-05-03 11:14:36,0.744363,0.498805


Найдем отношение активной и реактивной мощностей. Добавим его в качестве признака:

In [8]:
data_arc['power_ratio'] = data_arc['active_power'] / data_arc['reactive_power']

Вычисляем время начала нагрева и конца в формате UNIX:

In [9]:
data_arc['start_heating'] = (pd.to_datetime(data_arc['start_heating']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') 


In [10]:
data_arc['end_heating'] = (pd.to_datetime(data_arc['end_heating']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') 


Находим разницу во времени, то есть, время нагрева сплава:

In [12]:
data_arc['heating_time'] = data_arc['end_heating'] - data_arc['start_heating']


Группируем данные по номеру партии, и вычисляем:

- Суммарное время нагрева электродами, то есть сумму значений по всем промежуткам между запусками нагрева электродов.
- Количество запусков нагрева электродами.
- Среднее соотношение потребления активной и реактивной мощности.
- Статистики

In [13]:
data_arc_grouped = data_arc.groupby('key', as_index=False).agg(time_heating_total = ('heating_time', 'sum'),
                                                               num_heatings = ('start_heating', 'count'),
                                                               power_ratio_mean = ('power_ratio', 'mean'))

In [14]:
data_arc_grouped.head(3)

,key,time_heating_total,num_heatings,power_ratio_mean
0,1,1098,5,1.538898
1,2,811,4,1.551834
2,3,655,5,1.583120


Посмотрим на основные статистики:

In [15]:
data_arc_grouped.describe()

,key,time_heating_total,num_heatings,power_ratio_mean
count,3214.000000,3214.000000,3214.000000,3214.000000
mean,1623.061294,794.579029,4.628500,1.357452
std,934.739859,332.448622,1.608647,0.132824
min,1.000000,57.000000,1.000000,0.676106
25%,817.250000,571.000000,4.000000,1.292892
50%,1623.500000,770.000000,4.000000,1.370550
75%,2431.750000,983.000000,6.000000,1.438292
max,3241.000000,4189.000000,16.000000,1.782164


### 2. data_temp - данные о температуре:

Теперь найдем длительность времени между первым и последним замером температуры:

In [16]:
data_temp.head(3)

,key,measure_time,temperature
0,1,2019-05-03 11:16:18,1571.0
1,1,2019-05-03 11:25:53,1604.0
2,1,2019-05-03 11:29:11,1618.0


Для этого переведем время в удобный для вычислений формат. Сгруппируем данные по ключу. Найдем разницу между последним и первым вхождением времени:

In [17]:
data_temp['measure_time'] = (pd.to_datetime(data_temp['measure_time']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s') 


In [18]:
data_temp_grouped = data_temp.groupby('key', as_index=False).agg(first_measure_time = ('measure_time', 'first'),
                                                               last_measure_time = ('measure_time', 'last'))


In [19]:
data_temp_grouped['diff_bw_measures'] = data_temp_grouped['last_measure_time'] -\
                                        data_temp_grouped['first_measure_time']


Посмотрим на статистики:

In [20]:
data_temp_grouped['diff_bw_measures'].describe()

count     3216.000000
mean      1839.861629
std       1246.950298
min          0.000000
25%       1177.000000
50%       1581.000000
75%       2235.000000
max      23537.000000
Name: diff_bw_measures, dtype: float64